In [1]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 407 kB 12.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=a8b901700fa60fdfa35e3b7e12eeb7030bfe520872f5c8adde9c56723314b8bf
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.11.0+cu113
     |████████████████████████████████| 7.9 MB 10.8 MB/s 
     |████████████████████████████████| 3.5 MB 13.1 MB/s 


In [3]:
import os
import torch
import matplotlib.pyplot as plt

from torch_geometric.nn import GCNConv
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import NormalizeFeatures
import torch_geometric.transforms as T

transform = T.Compose([T.GCNNorm(), T.NormalizeFeatures()])

dataset = TUDataset(root="data/Proteins", name="PROTEINS", transform=None)
print(dataset)
data = dataset[0]
print(data)

Extracting data/Proteins/PROTEINS/PROTEINS.zip
Processing...


PROTEINS(1113)
Data(edge_index=[2, 162], x=[42, 0], y=[1])


Done!


In [47]:
print(f'Number of classes: {dataset.num_classes}')

Number of classes: 2


In [4]:
import numpy as np
torch.manual_seed(12345)
dataset = dataset.shuffle()

split = int(0.8 * len(dataset))
train_dataset, test_dataset = dataset[:split], dataset[split:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 890
Number of test graphs: 223


In [5]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 9268], x=[2464, 0], y=[64], batch=[2464], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 10700], x=[2919, 0], y=[64], batch=[2919], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 10964], x=[3004, 0], y=[64], batch=[3004], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 11064], x=[3063, 0], y=[64], batch=[3063], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 9094], x=[2395, 0], y=[64], batch=[2395], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 10592], x=[2732, 0], y=[64], batch=[2732], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 8866], x=[2326, 0], y=[64], batch=[2326], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 9324], x=[2478, 0], y=[64

In [6]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(0, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [7]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 31):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4011, Test Acc: 0.4170
Epoch: 002, Train Acc: 0.4011, Test Acc: 0.4170
Epoch: 003, Train Acc: 0.4011, Test Acc: 0.4170
Epoch: 004, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 005, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 006, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 007, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 008, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 009, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 010, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 011, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 012, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 013, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 014, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 015, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 016, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 017, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 018, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 019, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 020, Train Acc: 0.5989, Test Acc: 0.5830
Epoch: 021, Train Acc: 0.5989, Test Acc:

In [16]:
for data in test_loader:
  out = model(data.x, data.edge_index, data.batch) 
  print(F.softmax(out[0], dim=0)[0] > 2) 
  pred = out.argmax(dim=1) 
  print(pred[0].item()) 
  print(len(pred))
  break

tensor(False)
0
64


In [12]:
import os.path as osp
from re import sub
from sklearn import neighbors
import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, GNNExplainer

from torch_geometric.loader import DataLoader

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import subgraph
import numpy as np

import networkx as nx
from torch_geometric import utils

def expand(starts, ends, target, max_depth=1, prev=[]):
    node_neighbors = np.array([ends[idx] for idx, node in enumerate(starts) if node == target and ends[idx] != target])
    prev.append(target)
    if max_depth > 1:
        for n in node_neighbors:
            node_neighbors = np.concatenate((node_neighbors, expand(starts, ends, target=n, max_depth=max_depth-1, prev=prev)), axis=0)
    indices = np.unique(node_neighbors, return_index=True)[1]
    return np.array([node_neighbors[i] for i in sorted(indices)])


def process_one_graph(data):
    used = []
    num_nodes = data.x.shape[0]
    subgraph_sizes = [int(num_nodes/n) for n in range(2, 5)]
    start_nodes, end_nodes = np.array(data.edge_index)
    sub_graphs = []
    # make a grow from each node
    for target_idx in range(num_nodes):
        nodes_to_keep = expand(starts=start_nodes, ends=end_nodes, target=target_idx, max_depth=3, prev=[])
        if nodes_to_keep.shape[0] == 0:
            continue
        for size in subgraph_sizes:
            # select the grows based on size
            _subset = nodes_to_keep[:size]
            # remove repetitive stuff
            if set(_subset) in used:
                continue
            else:
                used.append(set(_subset))
            _subset = torch.from_numpy(np.array(_subset))
            sub_graphs.append((target_idx, data.subgraph(subset=_subset)))
    return sub_graphs



dataset = 'PROTEINS'
path = osp.join(osp.dirname(osp.realpath("__file__")), '..', 'data', 'TUDataset')
transform = T.Compose([T.GCNNorm(), T.NormalizeFeatures()])
dataset = TUDataset(path, dataset, transform=None)

pred_list = []
all_subgraphs = []
for idx, data in enumerate(tqdm.tqdm(dataset)):
    out = model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1) 
    pred_list.append(pred)

    subgraphs = process_one_graph(data)
    all_subgraphs.append(subgraphs)
    

# for target_idx, graph in tqdm.tqdm(all_subgraphs):
#     # nx.draw_networkx(utils.to_networkx(graph, remove_self_loops=True))
#     # plt.show()


Extracting /data/TUDataset/PROTEINS/PROTEINS.zip
Processing...
Done!
100%|██████████| 1113/1113 [01:33<00:00, 11.89it/s]


In [ ]:
print(type(all_subgraphs))
for data in all_subgraphs:
  print(data[1])
  break



<class 'list'>
(1, Data(edge_index=[2, 48], x=[14, 0], y=[1]))


In [ ]:
print(type(test_dataset))
for test_data in test_dataset:
  print(test_data)
  break

<class 'torch_geometric.datasets.tu_dataset.TUDataset'>
Data(edge_index=[2, 120], x=[28, 0], y=[1])


In [48]:
min_sufficient_explanation = [] #list of lists, each inner list correspond to one full graph in TUDataset Protein,
#containing each full graph's MSE's
threshold = 0.6
for idx, data_collection in enumerate(tqdm.tqdm(all_subgraphs)):
  sub_mse = [] #sub_mse for this collection, correspond to one original full graph
  for data in data_collection:
    out = model(data[1].x, data[1].edge_index, data[1].batch) 
    #use confidence 
    
    confidence = F.softmax(out[0], dim=0)
    pred = out.argmax(dim=1)
    #print(confidence)
    #print((confidence[pred.item()] >= threshold).item())
    
      
    # if the subgraph's predicted label matches the original full graph's predicted, add to its min_suff_expl list
    if torch.eq(pred, pred_list[idx]).item() and (confidence[pred.item()] >= threshold).item():
      sub_mse.append(data)
  
  min_sufficient_explanation.append(sub_mse)

  

100%|██████████| 1113/1113 [01:48<00:00, 10.21it/s]


In [45]:
for i in range(1110):
  #print(len(min_sufficient_explanation[i]))
  #print(len(all_subgraphs[i]))
  print(len(min_sufficient_explanation[i]) < len(all_subgraphs[i]))
for data in all_subgraphs:
  print(data)
  break

for data in min_sufficient_explanation:
  print(len(data))
  break

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
#input: 
# pred_list: [tensor, tensor, tensor ...], len = len(TUDataset/Protein)
#   each element is a one-element tensor 
#   representing the predicted label of one graph in TUDataset/Protein
# all_subgraph: [[subgraph, subgraph..], [subgraph, subgraph..]...], len = len(TUDataset/Protein)
#   each element is a list of subgraphs obtained from one graph in TUDataset/Protein
#   each subgraph is a tuple in the form (k, Data(edge_index=[2, 24], x=[8, 0], y=[1]))
#   where k is the index of the node that this subgraph expanded from
#output:
# min_sufficient_explanation: [[subgraph, subgraph,..], [subgraph, subgraph]...]
#   len = len(TUDataset/Protein)
#   each element is a list of subgraphs which are the MSEs
#   each subgraph is a tuple in the form (k, Data(edge_index=[2, 24], x=[8, 0], y=[1]))
#   where k is the index of the node that this subgraph expanded from